# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
E. Nasedkin  ->  E. Nasedkin  |  ['E. Nasedkin']
A. Dimoff  ->  A. Dimoff  |  ['A. Dimoff']
M. Ramirez  ->  M. Ramirez-Tannus  |  ['M. Ramirez']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']


Arxiv has 54 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.16363


extracting tarball to tmp_2409.16363...

 done.
  0: tmp_2409.16363/aassymbols.tex, 579 lines
  1: tmp_2409.16363/main.tex, 1,673 lines
Retrieving document from  https://arxiv.org/e-print/2409.16660
extracting tarball to tmp_2409.16660...

/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2409.16363/main.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

 done.
Retrieving document from  https://arxiv.org/e-print/2409.16916
extracting tarball to tmp_2409.16916...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2409.16916/SharkQ.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'AppendixSFHs' from 'tmp_2409.16916/AppendixSFHs.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'AppendixMainSequence' from 'tmp_2409.16916/AppendixMainSequence.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure Figs/SFHs_massive_passive_galaxies_z3_centrals_GALFORM.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[trim=4.5mm 4.5mm 2mm 4mm, clip,width=0.49\textwidth]{Figs/SFHs_massive_passive_galaxies_z3_centrals_GALFORM.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 117 bibliographic references in tmp_2409.16916/SharkQ.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.16660-b31b1b.svg)](https://arxiv.org/abs/2409.16660) | **The ESO SupJup Survey III: Confirmation of 13CO in YSES 1 b and Atmospheric Detection of YSES 1 c with CRIRES+**  |
|| Y. Zhang, et al. -- incl., <mark>P. Mollière</mark>, <mark>E. Nasedkin</mark> |
|*Appeared on*| *2024-09-26*|
|*Comments*| *25 pages, 11 figure, accepted for publication in AJ. The extracted CRIRES+ spectra of the YSES-1 system can be found at this https URL*|
|**Abstract**|            High-resolution spectroscopic characterization of young super-Jovian planets enables precise constraints on elemental and isotopic abundances of their atmospheres. As part of the ESO SupJup Survey, we present high-resolution spectral observations of two wide-orbit super-Jupiters in YSES 1 (or TYC 8998-760-1) using the upgraded VLT/CRIRES+ (R~100,000) in K-band. We carry out free atmospheric retrieval analyses to constrain chemical and isotopic abundances, temperature structures, rotation velocities, and radial velocities. We confirm the previous detection of 13CO in YSES 1 b at a higher significance of 12.6{\sigma}, but point to a higher 12CO/13CO ratio of 88+/-13 (1{\sigma} confidence interval), consistent with the primary's isotope ratio 66+/-5. We retrieve a solar-like composition in YSES 1 b with a C/O=0.57+/-0.01, indicating a formation via gravitational instability or core accretion beyond the CO iceline. Additionally, the observations lead to detections of H2O and CO in the outer planet YSES 1 c at 7.3{\sigma} and 5.7{\sigma}, respectively. We constrain the atmospheric C/O ratio of YSES 1 c to be either solar or subsolar (C/O=0.36+/-0.15), indicating the accretion of oxygen-rich solids. The two companions have distinct vsini, 5.34+/-0.14 km/s for YSES 1 b and 11.3+/-2.1 km/s for YSES 1 c, despite their similar natal environments. This may indicate different spin axis inclinations or effective magnetic braking by the long-lived circumplanetary disk around YSES 1 b. YSES 1 represents an intriguing system for comparative studies of super-Jovian companions and linking present atmospheres to formation histories.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.16761-b31b1b.svg)](https://arxiv.org/abs/2409.16761) | **S-Process Nucleosynthesis in Chemically Peculiar Binaries**  |
|| <mark>A. Dimoff</mark>, et al. |
|*Appeared on*| *2024-09-26*|
|*Comments*| *30 pages, 14 figures, 7 tables*|
|**Abstract**|            Around half of the heavy elements in the universe are formed through the slow neutron capture (s-) process, which takes place in thermally pulsing asymptotic giant branch (AGB) stars with masses $1-6\;M_{\odot}$. The nucleosynthetic imprint of the s-process can be studied by observing the material on the surface of binary barium, carbon, CH, and CEMP stars. We study the s-process by observing the luminous components of binary systems polluted by a previous AGB companion. Our radial velocity (RV) monitoring program establishes a collection of binary stars exhibiting enrichment in s-process material for the study of elemental abundances, production of s-process material, and binary mass transfer. From high resolution optical spectra, we measure RVs for 350 stars and derive stellar parameters for 150 stars using ATHOS. For a sub-sample of 24 stars we refine our atmospheric parameters using the Xiru program. We use the MOOG code to compute 1D-LTE abundances of C, Mg, s-process elements Sr, Y, Zr, Mo, Ba, La, Ce, Nd, Pb, and Eu to investigate neutron capture events and stellar chemical composition. We estimate dynamical masses by optimising orbits with MCMC techniques in the ELC program, and we compare our results with low-mass AGB models in the FRUITY database. We find enhancements in s-process material in spectroscopic binaries, a signature of AGB mass transfer. We add Mo to the abundance patterns, and for 12 stars we add Pb detections or upper limits. Computed abundances are in general agreement with the literature. Comparing our abundances to the FRUITY yields, we find correlations in s-process enrichment and AGB mass, and agreements in theoretical and dynamically modelled masses. From our high-resolution observations we expand heavy element abundance patterns and highlight binarity in our chemically interesting systems. We investigate evolutionary stages for a small sub-set of our stars.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.16916-b31b1b.svg)](https://arxiv.org/abs/2409.16916) | **The diverse star formation histories of early massive, quenched galaxies in modern galaxy formation simulations**  |
|| C. d. P. Lagos, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>L. Xie</mark> |
|*Appeared on*| *2024-09-26*|
|*Comments*| *Submitted for publication in MNRAS. 32 pages (21 of main body and 11 of appendices). Comments welcome!*|
|**Abstract**|            We present a comprehensive study of the star formation histories of massive-quenched galaxies at $z=3$ in 3 semi-analytic models (SHARK, GAEA, GALFORM) and 3 cosmological hydrodynamical simulations (EAGLE, Illustris-TNG, Simba). We study the predicted number density and stellar mass function of massive-quenched galaxies, their formation and quenching timescales and star-formation properties of their progenitors. Predictions are disparate in all these diagnostics, for instance: (i) some simulations reproduce the observed number density of very massive-quenched galaxies ($>10^{11}\rm M_{\odot}$) but underpredict the high density of intermediate-mass ones, while others fit well the lower masses but underpredict the higher ones; (ii) In most simulations, except for GAEA and EAGLE, most massive-quenched galaxies had starburst periods, with the most intense ones happening at $4<z<5$; however, only in SHARK and Illustris-TNG we do find a large number of progenitors with star formation rates $>300\rm M_{\odot}\,yr^{-1}$; (iii) quenching timescales are in the range $\approx 20-150$~Myr depending on the simulation; among other differences. These disparate predictions can be tied to the adopted Active Galactic Nuclei (AGN) feedback model. For instance, the explicit black-hole (BH) mass dependence to trigger the "radio mode" in Illustris-TNG and Simba makes it difficult to produce quenched galaxies with intermediate stellar masses, also leading to higher baryon collapse efficiencies ($\approx 15-30$%); while the strong bolometric luminosity dependence of the AGN outflow rate in GAEA leads to BHs of modest mass quenching galaxies. Current observations are unable to distinguish between these different predictions due to the small sample sizes. However, these predictions are testable with current facilities and upcoming observations, allowing a "true physics experiment" to be carried out.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.16363-b31b1b.svg)](https://arxiv.org/abs/2409.16363) | **Varstrometry for Off-nucleus and Dual sub-Kpc AGN (VODKA): A Mix of Singles, Lenses, and True Duals at Cosmic Noon**  |
|| A. C. Gross, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-09-26*|
|*Comments*| *26 pages, 8 figures, submitted to ApJ, includes appendix of 9 additional figures and 18 tables*|
|**Abstract**|            Dual Active Galactic Nuclei (dual AGNs), a phase in some galaxy mergers during which both central supermassive black holes (SMBHs) are active, are expected to be a key observable stage leading up to SMBH mergers. Constraining the population of dual AGNs in both the nearby and high-z universe has proven to be elusive until very recently. We present a multi-wavelength follow-up campaign to confirm the nature of a sample of 20 candidate dual AGNs at cosmic noon (z~2) from the VODKA sample. Through a combination of Hubble Space Telescope (HST) and Very Large Array (VLA) imaging, we refute the possibility of gravitational lensing in all but one target. We find evidence of dual AGNs in four systems, while seven exhibit single AGN in galaxy pairs, either through strong radio emission or ancillary emission line data. The remaining systems are either confirmed as quasar-star superpositions (six) or non-lensed pairs (two) that require further investigations to establish AGN activity. Among the systems with radio detections, we find a variety of radio spectral slopes and UV/optical colors suggesting that our sample contains a range of AGN properties, from obscured radio-quiet objects to those with powerful synchrotron-emitting jets. This study presents one of the largest dedicated multi-wavelength follow-up campaigns to date searching for dual AGNs at high redshift. We confirm several of the highest-z systems at small physical separations, thus representing some of the most evolved dual AGN systems at the epoch of peak quasar activity known to date.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.16890-b31b1b.svg)](https://arxiv.org/abs/2409.16890) | **The fast rise of the unusual Type IIL/IIb SN 2018ivc**  |
|| A. Reguitti, et al. -- incl., <mark>M. Ramirez</mark> |
|*Appeared on*| *2024-09-26*|
|*Comments*| *19 pages, 22 figures, 3 tables, 3 appendices, accepted for publication on A&A*|
|**Abstract**|            We present an analysis of the photometric and spectroscopic dataset of the Type II supernova (SN) 2018ivc in the nearby (10 Mpc) galaxy Messier 77. Thanks to the high cadence of the CHASE survey, we observed the SN rising very rapidly by nearly three magnitudes in five hours (or 18 mag d$^{-1}$). The $r$-band light curve presents four distinct phases: the maximum light is reached in just one day, then a first, rapid linear decline precedes a short-duration plateau. Finally, a long, slower linear decline lasted for one year. Following a radio rebrightening, we detected SN 2018ivc four years after the explosion. The early spectra show a blue, nearly featureless continuum, but the spectra evolve rapidly: after about 10 days a prominent H$\alpha$ line starts to emerge, with a peculiar profile, but the spectra are heavily contaminated by emission lines from the host galaxy. He I lines, namely $\lambda\lambda$5876,7065, are also strong. On top of the former, a strong absorption from the Na I doublet is visible, indicative of a non-negligible internal reddening. From its equivalent width, we derive a lower limit on the host reddening of $A_V\simeq1.5$ mag, while from the Balmer decrement and a match of the $B-V$ colour curve of SN 2018ivc to that of the comparison objects, a host reddening of $A_V\simeq3.0$ mag is obtained. The spectra are similar to those of SNe II, but with strong He lines. Given the peculiar light curve and spectral features, we suggest SN 2018ivc could be a transitional object between the Type IIL and Type IIb SNe classes. In addition, we found signs of interaction with circumstellar medium in the light curve, making SN 2018ivc also an interacting event. Finally, we modelled the early multi-band light curves and photospheric velocity of SN 2018ivc to estimate the explosion and CSM physical parameters.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2409.16660/figures/yses1b_res_ccf_CO_HF.png', 'tmp_2409.16660/figures/compare_corner_pt.png', 'tmp_2409.16660/figures/yses1b_bestfit_spec.png']
copying  tmp_2409.16660/figures/yses1b_res_ccf_CO_HF.png to _build/html/
copying  tmp_2409.16660/figures/compare_corner_pt.png to _build/html/
copying  tmp_2409.16660/figures/yses1b_bestfit_spec.png to _build/html/
exported in  _build/html/2409.16660.md
    + _build/html/tmp_2409.16660/figures/yses1b_res_ccf_CO_HF.png
    + _build/html/tmp_2409.16660/figures/compare_corner_pt.png
    + _build/html/tmp_2409.16660/figures/yses1b_bestfit_spec.png
found figures ['tmp_2409.16761/./figures/abundance_grid_specFe_2.png', 'tmp_2409.16761/./figures/FRUITY_model_comparison_full_3.png', 'tmp_2409.16761/./figures/cut_ATHOS_XIRU_sample_Kiel_FeH_3.png']
copying  tmp_2409.16761/./figures/abundance_grid_specFe_2.png to _build/html/
copying  tmp_2409.16761/./figures/FRUITY_model_comparison_full_3.png to _build/html/
copying  tmp_2409.16761/./

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand{\COiso}{^{13}\textrm{CO}}$
$\newcommand{\COmain}{^{12}\textrm{CO}}$
$\newcommand{\HTWOO}{H_{2}\textrm{O}}$
$\newcommand{\COTWO}{CO_{2}}$
$\newcommand{\methane}{CH_{4}}$
$\newcommand{\ammonia}{NH_{3}}$
$\newcommand{\enstatite}{MgSiO_{3}}$
$\newcommand{\Mjup}{M_\mathrm{Jup}}$
$\newcommand{\Rjup}{R_\mathrm{Jup}}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\Teff}{T_\mathrm{eff}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\rnote}[1]{{\textbf{\color{red}#1}}}$
$\newcommand{\leiden}{Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA, Leiden, The Netherlands}$
$\newcommand{\caltech}{Department of Astronomy, California Institute of Technology, Pasadena, CA 91125, USA}$
$\newcommand{\gps}{Division of Geological \& Planetary Sciences, California Institute of Technology, Pasadena, CA 91125, USA}$
$\newcommand{\warwick}{Department of Physics, University of Warwick, Coventry CV4 7AL, UK}$
$\newcommand{\warwicks}{Centre for Exoplanets and Habitability, University of Warwick, Gibbet Hill Road, Coventry CV4 7AL, UK}$
$\newcommand{\ipac}{IPAC, Mail Code 100-22, Caltech, 1200 E. California Boulevard, Pasadena, CA 91125, USA}$
$\newcommand{\mpia}{Max-Planck-Institut für Astronomie, Königstuhl 17, 69117 Heidelberg, Germany}$
$\newcommand{\granada}{Instituto de Astrof{í}sica de Andaluc{í}a (IAA-CSIC), Glorieta de la Astronom{í}a s/n, 18008 Granada, Spain}$
$\newcommand{\galway}{School of Natural Sciences, Center for Astronomy, University of Galway, Galway, H91 CF50, Ireland}$
$\newcommand{\carnegiew}{Earth and Planets Laboratory, Carnegie Institution for Science, Washington, DC, 20015}$</div>



<div id="title">

# The ESO SupJup Survey III: confirmation of $\COiso$ in YSES 1 b and atmospheric detection of YSES 1 c with CRIRES$^+$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.16660-b31b1b.svg)](https://arxiv.org/abs/2409.16660)<mark>Appeared on: 2024-09-26</mark> -  _25 pages, 11 figure, accepted for publication in AJ. The extracted CRIRES+ spectra of the YSES-1 system can be found at this https URL_

</div>
<div id="authors">

Y. Zhang, et al. -- incl., <mark>P. Mollière</mark>, <mark>E. Nasedkin</mark>

</div>
<div id="abstract">

**Abstract:** High-resolution spectroscopic characterization of young super-Jovian planets enablesprecise constraints on elemental and isotopic abundances of their atmospheres.As part of the ESO SupJup Survey, we present high-resolution spectral observations oftwo wide-orbit super-Jupiters in YSES 1 (or TYC 8998-760-1) using the upgraded VLT/CRIRES $^+$ ( $\mathcal{R}\sim 100,000$ )in K-band.We carry out free atmospheric retrieval analyses to constrain chemical and isotopic abundances, temperature structures,rotation velocities ( $v\sin i$ ), and radial velocities (RV).We confirm the previous detection of $\COiso$ in YSES 1 b at a higher significance of 12.6 $\sigma$ , but point to a higher $\COmain$ / $\COiso$ ratio of $88\pm 13$ (1 $\sigma$ confidence interval), consistent with the primary's isotope ratio $66 \pm 5$ .We retrieve a solar-like composition in YSES 1 b with a C/O $=0.57 \pm 0.01$ ,indicating a formation via gravitational instability or core accretion beyond the CO iceline.Additionally, the observations lead to detections of $\HTWOO$ and CO in the outer planet YSES 1 c at 7.3 $\sigma$ and 5.7 $\sigma$ , respectively.We constrain the atmospheric C/O ratio of YSES 1 c to be either solar or subsolar (C/O= $0.36 \pm 0.15$ ), indicating the accretion of oxygen-rich solids.The two companions have distinct $v\sin i$ , $5.34 \pm 0.14$ $\kms$ for YSES 1 b and $11.3 \pm 2.1$ $\kms$ for YSES 1 c, despite their similar natal environments.This may indicate different spin axis inclinations or effective magnetic braking by the long-lived circumplanetary disk around YSES 1 b.YSES 1 represents an intriguing system for comparative studies of super-Jovian companions and linking present atmospheres to formation histories.

</div>

<div id="div_fig1">

<img src="tmp_2409.16660/figures/yses1b_res_ccf_CO_HF.png" alt="Fig7" width="100%"/>

**Figure 7. -** Detection of $\COiso$ and HF molecules in CRIRES$^+$ observations of YSES 1 b.
      The top three panels show the 2.32-2.37 \micron observational residuals, where the main $\HTWOO$ and $\COmain$ spectral features have been removed from the data.
      The best-fit $\COiso$ and HF model spectra are overplotted in red and blue lines, demonstrating the detection of individual lines in the data.
      The bottom panels display cross-correlation functions (CCF) of observational residuals with $\COiso$ and HF models.
      The gray dashed line is each molecular model's auto-correlation function (ACF).
     (*fig:1b_ccf*)

</div>
<div id="div_fig2">

<img src="tmp_2409.16660/figures/compare_corner_pt.png" alt="Fig8" width="100%"/>

**Figure 8. -** Comparison of retrieval results of alternative models for YSES 1 b.
    Blue represents the baseline model, orange shows the enforced (optically thick) cloud model, green represents the model with veiling effect, and gray is the model with Gaussian Processes accounting for correlated noise.
    The upper right panel shows the retrieved T-P profiles. Different color saturations indicate the envelopes of
    1$\sigma$, 2$\sigma$, and 3$\sigma$ intervels. The red dashed line is the self-consistent T-P profile ($\Teff$=1900 K, $\log g$=4.0)
    from the Sonora Bobcat model grid. The dotted curve shows the flux-weighted emission contribution of each model.
    The corner plots show the posteriors of free parameters, including surface gravity, C/O ratio,
    metallicity, vertical mixing parameter, settling parameter, mass fraction of clouds at the cloud base, and carbon isotope ratio.
    We note the correlation between surface gravity and metallicity, while C/O and $\COmain$/$\COiso$ are less model dependent.
     (*fig:compare_pt*)

</div>
<div id="div_fig3">

<img src="tmp_2409.16660/figures/yses1b_bestfit_spec.png" alt="Fig6" width="100%"/>

**Figure 6. -** One spectral order of CRIRES$^+$ observations of YSES 1 b from 2.32 to 2.37 \micron.
    The observations are shown in black data points with error bars. Overplotted is the best-fit model
    obtained with retrieval analyses.
    We annotated the positions of absorption features from $\HTWOO$ and CO with short bars on the top of each panel.
    The observational residuals (data minus model) are shown with scattered dots in gray.
     (*fig:1b_spec*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.16660"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# S-Process Nucleosynthesis in Chemically Peculiar Binaries

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.16761-b31b1b.svg)](https://arxiv.org/abs/2409.16761)<mark>Appeared on: 2024-09-26</mark> -  _30 pages, 14 figures, 7 tables_

</div>
<div id="authors">

<mark>A. Dimoff</mark>, et al.

</div>
<div id="abstract">

**Abstract:** Around half of the heavy elements in the universe are formed through the slow neutron capture (s-) process, which takes place in thermally pulsing asymptotic giant branch (AGB) stars with masses $1-6\;M_{\odot}$ . The nucleosynthetic imprint of the s-process can be studied by observing the material on the surface of binary barium (Ba), carbon (C), CH, and carbon-enhanced metal-poor (CEMP) stars. We study the s-process by observing the luminous components of binary systems polluted by a previous AGB companion. Our radial velocity (RV) monitoring program establishes an ongoing collection of binary stars exhibiting enrichment in s-process material for the study of elemental abundances, production of s-process material, and binary mass transfer. From high resolution optical spectra, we measure radial velocities (RVs) for $350$ stars and derive stellar parameters for $\sim150$ stars using ATHOS. For a sub-sample of $24$ chemically interesting stars we refine our atmospheric parameters using ionization and excitation balance with the Xiru program. We use the MOOG code to compute one-dimensional local thermodynamic equilibrium (1D-LTE) abundances of carbon, magnesium, s-process elements (Sr, Y, Zr, Mo, Ba, La, Ce, Nd, Pb), and Eu to investigate neutron capture events and stellar chemical composition. We estimate dynamical stellar masses via orbital optimization using Markov chain Monte Carlo techniques in the ELC program, and we compare our results with low-mass AGB models in the FUll-Network Repository of Updated Isotopic Tables \& Yields (FRUITY) database. In our abundance sub-sample, we find enhancements in s-process material in spectroscopic binaries, a signature of AGB mass transfer. We add the element Mo to the abundance patterns, and for 12 stars we add Pb detections or upper limits, as these are not known in the literature. Computed abundances are in general agreement with the literature. Comparing our abundances to dilution-modified FRUITY yields, we find correlations in s-process enrichment and AGB mass, supported by dynamical modeling from RVs. From our high-resolution observations, we expand heavy element abundance patterns and highlight binarity in our chemically interesting systems. We find trends in s-process element enhancement from AGB stars, and agreements in theoretical and dynamically modelled masses. We investigate evolutionary stages for a small sub-set of our stars.

</div>

<div id="div_fig1">

<img src="tmp_2409.16761/./figures/abundance_grid_specFe_2.png" alt="Fig10" width="100%"/>

**Figure 10. -** Abundances for our sample of stars by element. Blue squares are Ba stars, red circles are carbon-enhanced (CEMP-s/-no or CH) stars, and green x's are "other" or unclassified stars. Inverted arrows are upper limits on the abundance of [X/Fe] (*fig:abund_grid*)

</div>
<div id="div_fig2">

<img src="tmp_2409.16761/./figures/FRUITY_model_comparison_full_3.png" alt="Fig12" width="100%"/>

**Figure 12. -** FRUITY models compared to computed abundances for our high-quality abundance sample, organized by decreasing metallicity. Blue data points correspond to Ba stars, red to C-enriched stars, and green to "other" stars. Inverted triangles in the plots are upper limits from our abundance computations. FRUITY model data is shown in black. (*fig:FRUITY_compare*)

</div>
<div id="div_fig3">

<img src="tmp_2409.16761/./figures/cut_ATHOS_XIRU_sample_Kiel_FeH_3.png" alt="Fig3" width="100%"/>

**Figure 3. -** TOP: Kiel diagram of our estimated stellar parameters from ATHOS and Xiru, with ATHOS operational limits as the dashed grey box. Open circles are parameters estimated by Xiru. Blue data points are known Ba stars, red are carbon enhanced stars, and green are "other" stars. Floating error bars are for Xiru (black) and ATHOS (green). Cool giants are in the upper right, and warm dwarfs are in the lower left. BOTTOM: Metallicity vs temperature for our this sub-sample, with ATHOS operational limits as the dashed grey box. Colors are the same as the top panel. (*fig:Kiel_Diag*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.16761"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\crate}{\dot M_{\rm cool}}$
$\newcommand{\eagle}{{\sc eagle}}$
$\newcommand{\surfs}{{\sc surfs}}$
$\newcommand{\shark}{{\sc Shark}}$</div>



<div id="title">

# The diverse star formation histories of early  massive, quenched galaxies in modern galaxy formation simulations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.16916-b31b1b.svg)](https://arxiv.org/abs/2409.16916)<mark>Appeared on: 2024-09-26</mark> -  _Submitted for publication in MNRAS. 32 pages (21 of main body and 11 of appendices). Comments welcome!_

</div>
<div id="authors">

C. d. P. Lagos, et al. -- incl., <mark>A. d. Graaff</mark>, <mark>L. Xie</mark>

</div>
<div id="abstract">

**Abstract:** We present a comprehensive study of the star formation histories of massive-quenched galaxies at $z=3$ in $3$ semi-analytic models ( $\shark$ , ${\sc GAEA}$ , ${\sc Galform}$ ) and $3$ cosmological hydrodynamical simulations ( ${\sc Eagle}$ , ${\sc Illustris-TNG}$ , ${\sc Simba}$ ). We study the predicted number density and stellar mass function of massive-quenched galaxies, their formation and quenching timescales and star-formation properties of their progenitors. Predictions are disparate in all these diagnostics, for instance: (i) some simulations reproduce the observed number density of very massive-quenched galaxies ( $>10^{11} \rm M_{\odot}$ ) but underpredict the high density of intermediate-mass ones, while others fit well the lower masses but underpredict the higher ones; (ii) In most simulations, except for ${\sc GAEA}$ and $\eagle$ , most massive-quenched galaxies had starburst periods, with the most intense ones happening at $4<z<5$ ; however, only in $\shark$ and ${\sc Illustris-TNG}$ we do find a large number of progenitors with star formation rates $>300\rm   M_{\odot} yr^{-1}$ ; (iii) quenching timescales are in the range $\approx 20-150$ Myr depending on the simulation; among other differences. These disparate predictions can be tied to the adopted Active Galactic Nuclei (AGN) feedback model. For instance, the explicit black-hole (BH) mass dependence to trigger the "radio mode" in ${\sc Illustris-TNG}$ and ${\sc Simba}$ makes it difficult to produce quenched galaxies with intermediate stellar masses, also leading to higher baryon collapse efficiencies ( $\approx 15-30$ \% ); while the strong bolometric luminosity dependence of the AGN outflow rate in ${\sc GAEA}$ leads to BHs of modest mass quenching galaxies. Current observations are unable to distinguish between these different predictions due to the small sample sizes. However, these predictions are testable with current facilities and upcoming observations, allowing a "true physics experiment’’ to be carried out.

</div>

<div id="div_fig1">

<img src="tmp_2409.16916/./Figs/max_sfr_mass_relations_sims_z3.png" alt="Fig13" width="100%"/>

**Figure 13. -** _ Top-left panel:_ maximum SFR achieved by the $z=3$ passive, massive galaxies (at any point of their history) as a function of the $z=3$ stellar mass in each of the $6$ simulations, as labelled. Lines with shaded regions show the medians and the $16^{\rm th}-84^{\rm th}$ percentile ranges, respectively. We use adaptive bins to ensure a reasonable number of galaxies per bin.
_ Top-right panel:_ as in the top-left panel but for the lookback time at which SFR$_{\rm max}$ happens. Here we show with horizontal lines three different redshifts, as labelled (noting that the cosmologies of the simulations are not exactly the same, but similar enough to use these lines as guidance). _ Bottom-left panel:_ Distance to the main sequence (i.e. $\rm log_{10}(sSFR/sSFR_{\rm MS})$; see \S \ref{definitions}), defined at the point of the maximum SFR of the $z=3$ massive-quenched galaxies as a function of the $z=3$ stellar mass. The dashed line shows the point at which galaxies are at $0.25$ dex above the main sequence. _ Bottom-right panel:_ As in the bottom-left panel but for the distance to main sequence in units of the width of the main sequence (measured as the standard deviation of the SFR-stellar mass relation at the point of the maximum SFR; see dotted lines in Fig. \ref{mainseq}). The dashed line shows the point at which galaxies are $1\sigma$ above the main sequence. (*sfr_scaling*)

</div>
<div id="div_fig2">

<img src="tmp_2409.16916/./Figs/num_density_passive_shark.png" alt="Fig11.1" width="16%"/><img src="tmp_2409.16916/./Figs/num_density_passive_eagle.png" alt="Fig11.2" width="16%"/><img src="tmp_2409.16916/./Figs/num_density_passive_gaea.png" alt="Fig11.3" width="16%"/><img src="tmp_2409.16916/./Figs/num_density_passive_tng.png" alt="Fig11.4" width="16%"/><img src="tmp_2409.16916/./Figs/num_density_passive_galform.png" alt="Fig11.5" width="16%"/><img src="tmp_2409.16916/./Figs/num_density_passive_simba.png" alt="Fig11.6" width="16%"/>

**Figure 11. -** Number density of passive massive galaxies selected based on two stellar mass thresholds, $M_{\star}>10^{10} \rm M_{\odot}$(thick lines) and $M_{\star}>10^{10.5} \rm M_{\odot}$(thin lines) and three sSFR thresholds, $<10^{-10} \rm yr^{-1}$(solid lines), $<5\times 10^{-11} \rm yr^{-1}$(dashed lines) and $<10^{-11} \rm yr^{-1}$(dotted lines), as labelled, between $2\le z\le 5$. This is shown for the $6$ simulations being analysed; the SAMs: $\shark${\sc GAEA}, and {\sc Galform}; and the hydrodynamical simulations: {\sc Eagle}, {\sc Illustris-TNG}, and {\sc Simba}, as labelled in each panel. Masses and SFRs are measured as described in \S \ref{definitions}.
For the redshift bins with $0$ passive galaxies, we use an arbitrary low number density of $10^{-9} \rm cMpc^{-3}$. Observational estimates are also shown: in grey symbols (pre-JWST results) are from [Straatman, et. al (2014)](), [Schreiber, et. al (2018)](), [Merlin, et. al (2019)](), [Girelli, Bolzonella and Cimatti (2019)](), [Carnall, et. al (2020)](), [Weaver, et. al (2023)](), [Gould, et. al (2023)](), and green symbols (JWST results) are from [Nanayakkara, et. al (2024)](), [Carnall, et. al (2023)](), [Valentino, et. al (2023)](), [Alberts, et. al (2023)](), as labelled in each panel. (*num_densities*)

</div>
<div id="div_fig3">

<img src="tmp_2409.16916/./Figs/stellarmf_passive_sims_z3.png" alt="Fig2" width="100%"/>

**Figure 2. -** _ Top panel:_ Stellar mass function of galaxies at $z=3$ selected to have a $\rm sSFR < 10^{-10} \rm yr^{-1}$ in the $6$ different simulations analysed, as labelled. We show Poisson errorbars with shaded regions to demonstrate that the differences between simulations are significant.
Observations from [Weaver, et. al (2023)]() for passive galaxies are shown in symbols for redshifts around $z=3$. The latter sample was selected using colour-colour selection (see text for details).
The vertical dotted line marks the approximate stellar mass above which observational estimates of the SMF are reliable according to [Weaver, et. al (2023)]().
_ Bottom panel:_ as in the top panel but after convolving stellar masses with Gaussian-distributed errors with a width of $0.25$ dex. (*SMFz3*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.16916"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

197  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

16  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
